In [ ]:
# Notebook criado para o desenvolvimento de envio de ordem do ativo PETR3
#
# Objetivo: Conexao com o MetaTrader 5, e envio de ordem baseados em indicadores
#            exemplo utilizado com o ativo PETR3

In [1]:
# Lendo as bibliotecas necessarias
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
from datetime import date
import MetaTrader5 as mt5
import time
# Biblioteca para indicadores tecnicos
import talib
from talib import RSI, EMA

In [2]:
# Conectando ao MetaTrader 5
# Utilizando os logins atuais do Software
if not mt5.initialize():
    print("initialize() falhou")
    mt5.shutdown()
# Consultamos o estado e os parâmetros de conexão
print(mt5.terminal_info())
# Obtemos informações sobre a versão do MetaTrader 5
print(mt5.version())

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=True, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=True, mqid=True, build=3300, maxbars=100000, codepage=1252, ping_last=71962, community_balance=0.0, retransmission=0.0, company='MetaQuotes Software Corp.', name='MetaTrader 5', language='Portuguese (Brazil)', path='C:\\Program Files\\MetaTrader 5', ...)
(500, 3300, '20 May 2022')


In [14]:
# Volume - Se não for múltiplo de 100, usar o ativo fracionado no simbolo
lote = 1
# Simbolo selecionado para grafico
simbolo = "PETR3F"

In [15]:
# Obtendo as informacoes do ativo de tamanho 1000, considerando a data de hoje e o Timframe de H1 = 1 hora;
ValoresAtivo = mt5.copy_rates_from_pos(simbolo, mt5.TIMEFRAME_H1, 0, 1000)
ValoresAtivo_PD = pd.DataFrame(ValoresAtivo) # Conversao em pandas
ValoresAtivo_PD['time'] = pd.to_datetime(ValoresAtivo_PD['time'], unit='s')

In [16]:
# Verificando um o tail() dos valores
ValoresAtivo_PD.tail()

,time,open,high,low,close,tick_volume,spread,real_volume
995,2022-05-23 13:00:00,38.57,38.63,38.47,38.61,315,1,2243
996,2022-05-23 14:00:00,38.63,38.85,38.56,38.85,322,1,2884
997,2022-05-23 15:00:00,38.81,39.10,38.81,39.04,332,1,3095
998,2022-05-23 16:00:00,39.09,39.27,38.95,39.16,477,1,3774
999,2022-05-23 17:00:00,38.62,39.25,38.62,39.02,97,0,1541


In [17]:
# Processando o Sinal
# Calculando o RSI, com 14 periodos, como parametro de entrada o valor de "close"
close = np.array(ValoresAtivo_PD['close'])
rsi_ind = RSI(close, timeperiod = 14)
array_length = len(rsi_ind)
last_element = rsi_ind[- 1]
last_rsi = last_element
array_length3 = len(close)
last_element3 = close[-1]
last_price = last_element3
price = mt5.symbol_info_tick(simbolo).ask
deviation = 20

In [18]:
# Estudar IC para sugerir como TP e SL

In [26]:
# Tomada de decisao e envio de ordem
# Se o ultimo valor do RSI for maior que 70, envia ordem de Venda

if last_rsi > 70  :
    request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": simbolo,
            "volume": lote,
            "type": mt5.ORDER_TYPE_SELL,
            "sl": price - 0.002,
            "tp": price + 0.005,
            "deviation": deviation,
            "magic": 202003,
            "comment": "BotPython - RSI",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC}
    print('Uma ordem de Venda foi enviada')
    mt5.order_send(request)
elif 30 > last_rsi :
    request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": simbolo,
            "volume": lote,
            "type": mt5.ORDER_TYPE_BUY,
            "sl": price + 0.002,
            "tp": price - 0.005,
            "deviation": deviation,
            "magic": 202003,
            "comment": "BotPython - RSI",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC}
    print('Uma ordem de Compra foi enviada')
    mt5.order_send(request)

else:
    print('Aguardand o sinal do Mercado')

Aguardand o sinal do Mercado
